# 6.1 Градиентный спуск и его виды


In [3]:
import seaborn as sns
import numpy as np
import pandas as pd
df = sns.load_dataset('diamonds')

In [4]:
# удалим лишние признаки
df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)
# закодируем оставшиеся признаки
df = pd.get_dummies(df, drop_first=True)

# логорифмируем признаки
df['carat'] = np.log(1+df['carat'])
df['price'] = np.log(1+df['price'])

# оперделим целевую переменную и предикток
X = df.drop(columns="price")
y = df["price"]

Разделите выборку на обучающую и тестовую (объём тестовой возьмите равным 0.33), значение random_state должно быть равно 42.

Теперь реализуйте алгоритм линейной регрессии со стохастическим градиентным спуском (класс SGDRegressor). Отберите с помощью GridSearchCV оптимальные параметры по следующей сетке:

In [5]:
from sklearn.model_selection import train_test_split
# Разделяем выборку на тренировочную и тестовую в соотношении 70/30
# Установим random_state для воспроизводимости результатов
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# Выведем результируюшие таблицы
print('Train: ', X_train.shape, y_train.shape)
print('Test: ', X_test.shape, y_test.shape)

Train:  (36139, 18) (36139,)
Test:  (17801, 18) (17801,)


In [9]:
# Создадим класс линейной регрессии c SGD
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

param = {"loss": ["squared_error", "epsilon_insensitive"],
"penalty": ["elasticnet"],
"alpha": np.logspace(-3, 3, 10),
"l1_ratio": np.linspace(0, 1, 10),
"learning_rate": ["constant"],
"eta0": np.logspace(-4, -1, 4)}

grid_search = GridSearchCV(
    estimator=linear_model.SGDRegressor(
        random_state=42, #генератор случайных чисел
        max_iter=1000 #количество итераций на сходимость
    ), 
    param_grid=param, 
    cv=5, 
    n_jobs = -1
)  

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

sgd = linear_model.SGDRegressor(**grid_search.best_params_, random_state = 42)

sgd.fit(X_train, y_train)
sgd.score(X_train, y_train) # r2
ls = sgd.predict(X_test)

round(mean_squared_error(y_test, ls), 3)

{'alpha': 0.001, 'eta0': 0.001, 'l1_ratio': 0.0, 'learning_rate': 'constant', 'loss': 'epsilon_insensitive', 'penalty': 'elasticnet'}


0.044

# 6.3 Метод Ньютона

In [8]:
# Задание 3.1

def func1(x):
    return 6*x**5-5*x**4-4*x**3+3*x**2
 
def func2(x):
    return 30*x**4-20*x**3-12*x**2+6*x
init_value = 0.7
iter_count = 0
x_curr = init_value
epsilon = 0.000001
f = func1(x_curr)
 
while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)
print(iter_count)

0.6296335078534031
0.6286680781673306
0.6286669787778999
0.6286669787764609
4


Оптимизация функции $f(x)=x^3-2*x^2-45*x+40$ при помощи метода ньютона

In [10]:
# Найдем первую и вторую производную
def func1(x):
    return 3*x**2 - 6*x -45
def func2(x):
    return 6*x - 6

Теперь необходимо взять какую-нибудь изначальную точку. Например, пусть это будет точка $x=42$. Также нам необходима точность — её возьмем равной 0.0001. На каждом шаге будем переходить в следующую точку по уже упомянутой выше формуле:

In [11]:
initial_value = 42
iter_count = 0
x_curr = initial_value
epsilon = 0.0001
f = func1(x_curr)

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)

21.695121951219512
11.734125501243229
7.1123493600499685
5.365000391507974
5.015260627016227
5.000029000201801
5.000000000105126
5.000000000000001


In [12]:
# Или можно сократить запись и написать как:
def newtons_method(f, fprime, x0, tol=0.0001):
    iter_count = 0
    x_curr = x0
    f_val = f(x_curr)
    while (abs(f_val) > tol):
        f_val = f(x_curr)
        f_prime_val = fprime(x_curr)
        x_curr = x_curr - (f_val)/(f_prime_val)
        iter_count += 1
    return x_curr

newtons_method(f=func1, fprime=func2, x0=50, tol=0.0001)

5.0

In [13]:
from scipy.optimize import newton
newton(func=func1, fprime=func2, x0=50, tol=0.0001)

5.0

In [14]:
# Задание 3.6

def func1(x):
    return x**3 -72*x -220
 
def func2(x):
    return 3*x**2 -72
init_value = 12
iter_count = 0
x_curr = init_value
epsilon = 0.000001
f = func1(x_curr)
 
while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)
print(iter_count)

10.21111111111111
9.756461564762278
9.727252176332618
9.72713442131889
9.727134419408875
5


In [15]:
# Задание 3.7

def func1(x):
    return x**2+9*x - 5
 
def func2(x):
    return 2*x+9
init_value = 2.2
iter_count = 0
x_curr = init_value
epsilon = 0.000001
f = func1(x_curr)
 
while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)
print(iter_count)

0.7343283582089555
0.5291259698087832
0.5249395544696249
0.5249378105607477
0.5249378105604451
5


In [16]:
# Task 3.9
def func1(x):
    return 24*x**2 - 4*x
 
def func2(x):
    return 48*x -4

def newtons_method(f, fprime, x0, tol=0.0001):
    iter_count = 0
    x_curr = x0
    f_val = f(x_curr)
    while (abs(f_val) > tol):
        f_val = f(x_curr)
        f_prime_val = fprime(x_curr)
        x_curr = x_curr - (f_val)/(f_prime_val)
        iter_count += 1
    return x_curr

newtons_method(f=func1, fprime=func2, x0=42, tol=0.0001)

0.16666666666666785

# 6.4 Квазиньютоновские методы

In [1]:
import numpy as np
from scipy.optimize import minimize

Давайте рассмотрим, как с помощью функций Python мы сможем применить квазиньютоновские методы для оптимизации функции $f(x,y)=x^2+y^2$ 

In [2]:
# Определим функцию, которую будем оптимизировать. Вместо отдельных x и y можно взять координаты единого вектора:

def func(x):
    return x[0]**2.0 + x[1]**2.0

# Теперь определим градиент для функции:
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])

# Зададим начальную точку
x_0 =[1.0, 1.0]

# Определим алгоритм

result = minimize(func, x_0, method='BFGS', jac=grad_func)

print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


In [3]:
# Можно повторить то же самое с вариацией  L-BFGS-B:

# определяем нашу функцию
def func(x):
    return x[0]**2.0 + x[1]**2.0
 
#  определяем градиент функции
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])
 
# определяем начальную точку
x_0 = [1, 1]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


In [5]:
# Task 4.1

def func1(x):
    return x[0]**2 -x[0]*x[1] + x[1]**2 + 9*x[0] - 6*x[1] + 20

def grad_func1(x):
    return np.array([2*x[0] - x[1] + 9, -x[0] + 2*x[1] - 6])

x_0 = [-400, -400]


# реализуем алгоритм L-BFGS-B
result = minimize(func1, x_0, method='L-BFGS-B', jac=grad_func1)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func1(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))


Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 9
Решение: f([-3.99999972  1.00000028]) = -1.00000


In [8]:
# Task 4.4
# Определим функцию, которую будем оптимизировать. Вместо отдельных x и y можно взять координаты единого вектора:

def func(x):
    return x**2 - 3*x +45

# Теперь определим градиент для функции:
def grad_func(x):
    return 2*x - 3

# Зададим начальную точку
x_0 =[10]

# Определим алгоритм

result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)

print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([1.5]) = 42.75000


In [9]:
# TAsk 4.7

def func1(x):
    return x[0]**4 + 6*x[1]**2 + 10

def grad_func1(x):
    return np.array([4*x[0]**3, 12*x[1]])

x_0 = [100, 100]


# реализуем алгоритм L-BFGS-B
result = minimize(func1, x_0, method='L-BFGS-B', jac=grad_func1)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func1(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

result_1 = minimize(func1, x_0, method='BFGS', jac=grad_func1)
# получаем результат
print('Статус оптимизации %s' % result_1['message'])
print('Количество оценок: %d' % result_1['nfev'])
solution = result_1['x']
evaluation = func1(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Количество оценок: 40
Решение: f([-9.52718297e-03 -2.32170510e-06]) = 10.00000
Статус оптимизации Optimization terminated successfully.
Количество оценок: 37
Решение: f([1.31617159e-02 6.65344582e-14]) = 10.00000


# 6.6 Практика линейного програмирования

В языке Python есть множество библиотек, с помощью которых можно решить задачу линейного программирования. Вот основные, которые мы рассмотрим в данном юните:

- SciPy (scipy.optimize.linprog);
- CVXPY; 
- PuLP.

У нас есть 6 товаров с заданными ценами на них и заданной массой.

Вместимость сумки, в которую мы можем положить товары, заранее известна и равна 15 кг.

Какой товар и в каком объёме необходимо взять, чтобы сумма всех цен товаров была максимальной?

In [1]:
import numpy as np

# Создадим переменные
values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров

# Зададим вектора
c = - np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива
b = np.array([C]) #конвертируем вместимость в массив

from scipy.optimize import linprog
linprog(c=c, A_ub=A, b_ub=b)



           con: array([], dtype=float64)
 crossover_nit: 0
         eqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
           fun: -52.5
       ineqlin:  marginals: array([-3.5])
  residual: array([0.])
         lower:  marginals: array([13.5, 29.5, 27. ,  0. , 18. , 11.5])
  residual: array([0. , 0. , 0. , 7.5, 0. , 0. ])
       message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
           nit: 0
         slack: array([0.])
        status: 0
       success: True
         upper:  marginals: array([0., 0., 0., 0., 0., 0.])
  residual: array([inf, inf, inf, inf, inf, inf])
             x: array([0. , 0. , 0. , 7.5, 0. , 0. ])

In [2]:
import cvxpy
x = cvxpy.Variable(shape=n, integer = True) # С помощью CVXPY создадим переменную-массив. 
                                            #Укажем его размерность, а также условие, что все числа в массиве должны быть целыми
                                            
A = A.flatten() # Преобразуем размерность массива
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
total_value = cvxpy.sum(cvxpy.multiply(x, c))

x = cvxpy.Variable(shape=n, integer=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0 # необходимо чтоб ответ не ушел в бесконечность
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
x.value

array([-0., -0., -0.,  7., -0.,  0.])

In [4]:
# если можно брать не целые значения 

x = cvxpy.Variable(shape=n, boolean=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
x.value


array([1., 0., 0., 1., 0., 1.])

В нашей каршеринговой компании две модели автомобилей: модель *А* и модель *В*. Автомобиль *А* даёт прибыль в размере 20 тысяч в месяц, а автомобиль *В* — 45 тысяч в месяц. Мы хотим заказать на заводе новые автомобили и максимизировать прибыль. Однако на производство и ввод в эксплуатацию автомобилей понадобится время:

- Проектировщику требуется 4 дня, чтобы подготовить документы для производства каждого автомобиля типа *А*, и 5 дней — для каждого автомобиля типа *В*.
- Заводу требуется 3 дня, чтобы изготовить модель *А*, и 6 дней, чтобы изготовить модель *В*.
- Менеджеру требуется 2 дня, чтобы ввести в эксплуатацию в компании автомобиль *А*, и 7 дней —  автомобиль *В*.
- Каждый специалист может работать суммарно 30 дней.

In [6]:
from pulp import *

problem = LpProblem('Производство машин', LpMaximize)
A = LpVariable('Автомобиль A', lowBound=0 , cat=LpInteger)
B = LpVariable('Автомобиль B', lowBound=0 , cat=LpInteger)

#Целевая функция
problem += 20000*A + 45000*B 
#Ограничения

problem += 4*A + 5*B <= 30 
problem += 3*A + 6*B <=30
problem += 2*A + 7*B <=30
problem.solve()


print("Количество автомобилей модели А: ", A.varValue)
print("Количество автомобилей модели В: ", B.varValue)
print("Суммарный доход: ", value(problem.objective))

Количество автомобилей модели А:  1.0
Количество автомобилей модели В:  4.0
Суммарный доход:  200000.0


In [12]:
# Task 6.1

problem = LpProblem('Производство машин', LpMinimize)
x_1 = LpVariable('x_1', lowBound=0 , cat=LpInteger)
x_2 = LpVariable('x_2', lowBound=0 , cat=LpInteger)
x_3 = LpVariable('x_3', lowBound=0 , cat=LpInteger)
y_1 = LpVariable('y_1', lowBound=0 , cat=LpInteger)
y_2 = LpVariable('y_2', lowBound=0 , cat=LpInteger)
y_3 = LpVariable('y_3', lowBound=0 , cat=LpInteger)


#Целевая функция
problem += 2*x_1 + 5*x_2 + 3* x_3 + 7*y_1 + 7*y_2 + 6*y_3 
#Ограничения

problem += x_1 + x_2 + x_3 <= 180 
problem += y_1 + y_2 + y_3 <= 220 
problem += x_1 + y_1 >= 110
problem += x_2 + y_2 >= 150
problem += x_3 + y_3 >= 140
problem.solve()


print("Количество товара на складе x_1: ", x_1.varValue)
print("Количество товара на складе x_2: ", x_2.varValue)
print("Количество товара на складе x_3: ", x_3.varValue)
print("Количество товара на складе y_1: ", y_1.varValue)
print("Количество товара на складе y_2: ", y_2.varValue)
print("Количество товара на складе y_3: ", y_3.varValue)
print("Суммарный доход: ", value(problem.objective))

Количество товара на складе x_1:  110.0
Количество товара на складе x_2:  0.0
Количество товара на складе x_3:  70.0
Количество товара на складе y_1:  0.0
Количество товара на складе y_2:  150.0
Количество товара на складе y_3:  70.0
Суммарный доход:  1900.0


c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [13]:
# Task 6.2
import numpy as np
import cvxpy

# Задаем матрицу A, где A[i, j] - затраты на выполнение задачи i исполнителем j
A = np.array([[1000, 12, 10, 19, 8],
              [12, 1000, 3, 7, 2],
              [10, 3, 1000, 6, 20],
              [19, 7, 6, 1000, 4],
              [8, 2, 20, 4, 1000]])

# Определяем бинарные переменные
x = cvxpy.Variable(shape=(5, 5), boolean=True)

# Определяем переменную y, которая является транспонированной матрицей x
y = x.T

# Определяем ограничения
x_0 = cvxpy.sum(x[0]) >= 1
x_1 = cvxpy.sum(x[1]) >= 1
x_2 = cvxpy.sum(x[2]) >= 1
x_3 = cvxpy.sum(x[3]) >= 1
x_4 = cvxpy.sum(x[4]) >= 1

# Каждый исполнитель должен быть назначен хотя бы на одну задачу
# Сумма элементов каждого столбца матрицы y должна быть больше или равна 1
z0 = cvxpy.sum(y[0]) >= 1
z1 = cvxpy.sum(y[1]) >= 1
z2 = cvxpy.sum(y[2]) >= 1
z3 = cvxpy.sum(y[3]) >= 1
z4 = cvxpy.sum(y[4]) >= 1

# Определяем целевую функцию, которую нужно минимизировать
# Суммарные затраты на работы
total_value = cvxpy.sum(cvxpy.multiply(x, A))

# Определяем задачу оптимизации
problem = cvxpy.Problem(
    cvxpy.Minimize(total_value),
    constraints=[
        cvxpy.sum(x, axis=0) == np.ones(5),
        cvxpy.sum(x, axis=1) == np.ones(5)
    ]
)

# Решаем задачу оптимизации
result = problem.solve()

# Выводим оптимальное значение функции (минимальные затраты)
print(result)

# Получаем значения переменных x после решения задачи
x_values = x.value

# Выводим значения переменных x
print(x_values)

32.0
[[ 0.  0. -0.  0.  1.]
 [ 0.  0.  1.  0.  0.]
 [ 1. -0.  0.  0.  0.]
 [ 0.  1.  0.  0. -0.]
 [-0.  0.  0.  1.  0.]]
